# Step1: Initializing Setup

In [1]:
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import cv2
import os
import sys
import albumentations as aug
from model import RowDetection
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import time
from tqdm import tqdm_notebook
import matplotlib.image as mpimg

from train import *
from dataloader import *
from visualize import *

import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
print(device)

C:\Users\ALI\.conda\envs\LW_Unet_PT\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.5 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


cuda


# Step 2: Augmenting and Visualizating Data

In [2]:

images = "C:\\Users\\ALI\\Documents\\Ali_Thesis\\Algo2\\Pretrained_UNet\\train_data\\image\\"
masks = "C:\\Users\\ALI\\Documents\\Ali_Thesis\\Algo2\\Pretrained_UNet\\train_data\\mask\\"

train_dir = images + "train\\"
val_dir = images + "val\\"
train_images = sorted(os.listdir(train_dir))
val_images = sorted(os.listdir(val_dir))

print(len(val_images))
print(len(train_images))


#---------------Apply augmentations---------------------

train_trans = [aug.RandomRotate90(p=0.5),aug.Flip(p=0.5),aug.Resize(384,384),
                   #aug.IAAAdditiveGaussianNoise(p=0.2),aug.IAAPerspective(p=0.5),
                   #aug.OneOf([aug.CLAHE(p=1),aug.RandomBrightness(p=1),aug.RandomGamma(p=1)],p=0.9),
                   #aug.OneOf([aug.RandomContrast(p=1),aug.HueSaturationValue(p=1),],p=0.9)
              ]
              
#train_dataset = DatasetLoader(train_images, train_dir,masks)
train_dataset = DatasetLoader(train_images, train_dir,masks,transform=train_trans)

val_trans = [aug.Resize(384,384)]

#val_dataset = DatasetLoader(val_images, val_dir, masks)
val_dataset = DatasetLoader(val_images, val_dir, masks,transform=val_trans)

'''
#---------------visualization of an image (uncomment if needed)-------------------
for i in range(len(train_dataset)):
    image, mask = train_dataset[i]
    visualize(image=image, mask=mask)
'''


16
125


C:\Users\ALI\AppData\Local\Temp\ipykernel_17224\329326571.py:15: DeprecationWarning: Flip is deprecated. Consider using HorizontalFlip, VerticalFlip, RandomRotate90 or D4.
  train_trans = [aug.RandomRotate90(p=0.5),aug.Flip(p=0.5),aug.Resize(384,384),


'\n#---------------visualization of an image (uncomment if needed)-------------------\nfor i in range(len(train_dataset)):\n    image, mask = train_dataset[i]\n    visualize(image=image, mask=mask)\n'

# Step 3: Initializing Training Paramerters

In [3]:
batch_size = 8
train_loader = DataLoader(train_dataset,batch_size = batch_size,shuffle = True)
val_loader = DataLoader(val_dataset,batch_size = batch_size,shuffle=False)

print(len(train_loader))
print(len(val_loader))

mymodel = RowDetection(3,1)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(mymodel.parameters(),lr=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode='min',factor=0.2,patience=5)

#------------------------Initialize model---------------------
mymodel.load_state_dict(torch.load("C:\\Users\\ALI\Documents\\Ali_Thesis\\Algo2\\Pretrained_UNet\\weights\\LED_Matrix_Weights.pt"))

torch.autograd.set_detect_anomaly(True)
#nn.ReLU(inplace=False)

16
2


C:\Users\ALI\.conda\envs\LW_Unet_PT\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ALI\.conda\envs\LW_Unet_PT\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


# Step 4: Training

In [4]:
#----------------------Train loop----------------------

epochs = 100
Train_loss = []
Val_loss = []
for i in range(epochs):
    print("Epochs : ",(i+1),"\\",(epochs))
    train_loss = train_epoch(mymodel, train_loader, criterion,optimizer, device)
    val_loss = eval_model(mymodel, val_loader,criterion, device)
    Train_loss.append(train_loss)
    Val_loss.append(val_loss)
    print("--"*30)
    scheduler.step(val_loss)
print('Training done...')    
#-----------------Save last weights-------------------
torch.save(mymodel.state_dict(), 'C:\\Users\\ALI\\Documents\\Ali_Thesis\\Algo2\\Pretrained_UNet\\weights\\LED_Matrix_Weights.pt')


Epochs :  1 \ 100


  0%|          | 0/16 [00:00<?, ?it/s]

Training Loss:  0.039672860759310424


  0%|          | 0/2 [00:00<?, ?it/s]

Testing Loss:  0.03373649995774031
------------------------------------------------------------
Epochs :  2 \ 100


...

Testing Loss:  0.002534217550419271
------------------------------------------------------------
Epochs :  100 \ 100


  0%|          | 0/16 [00:00<?, ?it/s]

Training Loss:  0.002626924979267642


  0%|          | 0/2 [00:00<?, ?it/s]

Testing Loss:  0.002536078216508031
------------------------------------------------------------
Training done...


In [5]:
torch.save(mymodel.state_dict(), 'C:\\Users\\ALI\\Documents\\Ali_Thesis\\Algo2\\Pretrained_UNet\\weights\\LED_Matrix_Weights.pt')
